# Car Quality Metrics

## Estimate the Quality using the Loaded Classifier

In [1]:
import os
import glob
import PIL.Image
import numpy as np
import torch
from car_quality_estimator.car_quality_metric import load_car_quality_score

test_image_dir = "example_data/2c21b97ff3dc4fc3b1ef9e4bb0164318"
car_quality_metric = load_car_quality_score(use_combined_embedding_model=True)

In [2]:
# load all images from the test directory and store the PIL images in a list
all_images = glob.glob(os.path.join(test_image_dir, "*.png"))
all_images = [PIL.Image.open(image).convert("RGB") for image in all_images]
reference_batch = all_images

# let's generate some distorted views using Gaussian noise
def create_noised_image(image_list):
    all_images_np = np.array(image_list) / 255.0
    all_images_distorted_np  = all_images_np + np.random.randn(*all_images_np.shape) * 0.05
    all_images_distorted_np = np.clip(all_images_distorted_np, 0, 1)
    all_images_distorted_pil = [PIL.Image.fromarray((img * 255).astype(np.uint8)) for img in all_images_distorted_np]
    return all_images_distorted_pil
generated_views = create_noised_image(reference_batch)

In [ ]:
# Quality score with reference images
car_quality_metric.compute_scores_no_reference(generated_views)

In [ ]:
# Quality score with reference images
car_quality_metric.compare_with_reference(
    generated_views, reference_batch, compute_kid=True)